# Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import copy

## Import skeleton from plantscan3d as .txt file (MTG)

In [3]:
# Loading in data
filename = 'image_4.txt' #put your filename here

# setting everything up nicely

my_array = pd.read_csv(filename,header=0,sep='\t')
my_array['Parent'].loc[0] = 0
my_array['Parent'] = my_array['Parent'].astype('int')
my_array['Node'] = my_array['Node'].astype('int')
# visualize

my_array

/Users/jiaxinyang/anaconda3/envs/plant/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Node,Parent,X,Y,Z
0,2,0,-0.009948,-0.069728,0.038775
1,3,2,-0.009381,-0.063063,0.036603
2,4,3,-0.009509,-0.049572,0.028704
3,7,4,-0.008002,-0.035813,0.023481
4,13,7,-0.005734,-0.024126,0.017773
5,20,13,-0.011288,-0.009588,0.001509
6,21,13,0.005039,-0.012246,0.007310
7,22,3,-0.027297,-0.005240,0.012878
8,27,20,-0.003285,0.005474,-0.003666
9,30,27,0.000145,0.021746,-0.005362


## Functions of calculate curvatures 

In [20]:
def ratio_cal(my_array, branch_list):
    '''
    curvature = geodesic distance / euclidean distance
    :parm  skeleton data frame
    :parm  list of node id of one branch
    '''
    # get the coordinate list
    coord_list = []
    for node_id in branch_list:
        coord_list.append(my_array[my_array['Node'] == node_id].values[0][2:])
    
    # euclidean distance between two end points
    euclid_dist = np.linalg.norm(coord_list[0] - coord_list[-1])
    
    # geodesic distance
    geo_dist = 0
    for first_id in range(len(coord_list)-1):
        geo_dist += np.linalg.norm(coord_list[first_id] - coord_list[first_id+1])
        
    return geo_dist/euclid_dist

In [21]:
def curvature_cal(my_array):
    '''
    find the branch and calculate curvatures
    :parm  skeleton data frame
    '''
    
    parents = my_array['Parent'] # getting all of the parent nodes
    curvatures = [] # list of curvatures for final output
    tested = [] #list of parents already tested

    for parent in parents: # for each parent in the list
        children = my_array[my_array['Parent'] == parent] # retrieve rows of dataframe that list the parent node as their parent
        if len(children) >= 2 and parent not in tested: # if that's 2 or greater, it must be a branching point
            for first_child in children['Node']:
                # get the list of branch
                parent_to_child_vectors = [parent]
                prev_child = first_child
                parent_to_child_vectors.append(prev_child)
                next_children = my_array[my_array['Parent'] == prev_child]
                while len(next_children) == 1: # to the end point of a branch
                    prev_child = next_children['Node'].values[0]
                    parent_to_child_vectors.append(prev_child)
                    next_children = my_array[my_array['Parent'] == prev_child]
                if len(parent_to_child_vectors) > 2: # only consider the branch with at least 3 points
                    curvatures.append(ratio_cal(my_array, parent_to_child_vectors))
    return curvatures

## Print results

In [22]:
curvature_cal(my_array)

[1.005204488249097,
 1.0754582321316177,
 1.0266958348322295,
 1.0266958348322295,
 1.005204488249097,
 1.0754582321316177,
 1.0142346168319807,
 1.0280107905708884,
 1.021542156906465,
 1.087820177397586,
 1.021542156906465,
 1.087820177397586,
 1.0142346168319807,
 1.0280107905708884,
 1.146532346564532,
 1.0284484807357823,
 1.146532346564532,
 1.0284484807357823,
 1.008220172674479,
 1.0519741903450652,
 1.008220172674479,
 1.0519741903450652]